圖片來源:https://data.mendeley.com/datasets/7w7fccy7ky/4

In [34]:
import os
import cv2
from ultralytics import YOLO
import csv
conf = 0.3
s = 1
dir = "Fall forward"
# Path to the directory containing images
input_dir = f'C:/Users/User/Desktop/Dataset CAUCAFall/CAUCAFall/Subject.{s}/{dir}' # 存放image資料夾
output_dir = f'C:/Users/User/Desktop/Dataset CAUCAFall/CAUCAFall/Subject.{s}/{dir}_detect_{conf}' # 偵測後輸出image 資料夾
output_file = f"C:/Users/User/Desktop/Dataset CAUCAFall/keypoints_data_s{s}_{dir}_{conf}.csv" # image keypoint csv
# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Load YOLOv8 model for pose detection
model = YOLO('yolov8n-pose.pt')  # Ensure you have the correct YOLOv8 pose model
def process_image(results):
    r = results[0]
    combined_results = []
    for i in range(len(r.keypoints.xyn)):
        for  j  in range(17):
            x, y = r.keypoints.xyn[i][j].numpy()
            confidence = r.keypoints.conf[i][j].item()  # Convert tensor to float
            combined_results.append((x, y, confidence))
    return combined_results
# Process each JPG image in the input directory
all_keypoints_data = []
for idx, filename in enumerate(os.listdir(input_dir)):
    if filename.lower().endswith('.png'):
        input_path = os.path.join(input_dir, filename)
        
        # New filename with renaming convention, e.g., prefix "detected_" + original filename
        new_filename = f'detected_{idx:04d}.png'  # e.g., detected_0001.jpg, detected_0002.jpg, etc.
        output_path = os.path.join(output_dir, new_filename)

        # Load the image
        image = cv2.imread(input_path)
        # resized image to 640*640
        # resized_image = cv2.resize(image, (640, 640))
        # Perform pose detection
        results = model(image, conf=conf)

        if len(results[0].boxes) == 1:
            # Draw the results on the image
            annotated_image = results[0].plot()
            # Save the annotated image with the new filename
            cv2.imwrite(output_path, annotated_image)
            print(f"Processed and saved: {output_path}")
            keypoints_data = process_image(results)
            keypoints_data.insert(0, new_filename)
            all_keypoints_data.append(keypoints_data)
            
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # Write the header
    header = ["image_name"] + [f"x{i+1}, y{i+1}, confidence{i+1}" for i in range(len(all_keypoints_data[0]) - 1)]
    writer.writerow(header)
    
    # Write the data
    writer.writerows(all_keypoints_data)

print("Data saved to", output_file)


0: 448x640 1 person, 98.6ms
Speed: 3.0ms preprocess, 98.6ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)
Processed and saved: C:/Users/User/Desktop/Dataset CAUCAFall/CAUCAFall/Subject.1/Fall forward_detect_0.3\detected_0000.png

0: 448x640 1 person, 93.5ms
Speed: 2.0ms preprocess, 93.5ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)
Processed and saved: C:/Users/User/Desktop/Dataset CAUCAFall/CAUCAFall/Subject.1/Fall forward_detect_0.3\detected_0002.png

0: 448x640 1 person, 94.3ms
Speed: 1.1ms preprocess, 94.3ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
Processed and saved: C:/Users/User/Desktop/Dataset CAUCAFall/CAUCAFall/Subject.1/Fall forward_detect_0.3\detected_0004.png

0: 448x640 2 persons, 95.4ms
Speed: 1.0ms preprocess, 95.4ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 person, 98.8ms
Speed: 3.0ms preprocess, 98.8ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)


KeyboardInterrupt: 